In [1]:
import csv
import os
from datetime import datetime
import math
import pandas as pd


In [2]:
def print_table(table):
    for row in table:
        print(row)

In [3]:
def create_table(tab1, tab2, tab3):           #optimized version
    result_tab = []

    emotions = ['Unknown', 'Happy', 'Sad', 'Scared', 'Disgusted', 'Surprised', 'Angry']

    for row1, row2, row3 in zip(tab1, tab2, tab3):
        sum_rows = [int(a) + int(b) + int(c) for a, b, c in zip(row1[1:], row2[1:], row3[1:])]

        total_sum = sum(sum_rows)
        if total_sum > 0:
            # Calculate the percentage distribution
            percentages = [round((x / total_sum) * 100, 2) for x in sum_rows]
        else:
            percentages = [0] * len(sum_rows)

        result_tab.append([row1[0]] + percentages)

    return result_tab

In [4]:
def fill_table(table,csvreader):
    seconds = 0
    stop = 0
    for row in csvreader:
        row[0] = int(row[0].split('.')[0])
        if seconds <= row[0] and stop == 0:     #fill table with missing seconds
            for i in range(0,row[0]):
                table.append([i, '0', '0', '0', '0', '0', '0', '0'])
                seconds+=1
            stop = 1
            table.append(row)
        else:
            table.append(row)

In [6]:
def csv_files_reader(base_path):#read csv files from Boris
    index = ['I','II','III']
    tab1 = []
    tab2 = []
    tab3 = []

    if os.path.exists(base_path):

        for i in index:
            for file_name in os.listdir(base_path+i):
                file_path = os.path.join(base_path+i, file_name)

                with open(file_path, 'r') as file:
                    csvreader = csv.reader(file)
                    header = next(csvreader)

                    if i == 'I':
                        fill_table(tab1,csvreader)
                    elif i == 'II':
                        fill_table(tab2,csvreader)
                    elif i == 'III':
                        fill_table(tab3,csvreader)
    else:
        print(f"File path {base_path} doesn't exist.")

    large_table = max([tab1, tab2, tab3], key=len)

    for tab in [tab1, tab2, tab3]:
        if len(tab) < len(large_table):
            diff = len(large_table) - len(tab)

            if len(tab) == 0:
                sec=0
                for sec in range(len(large_table)):
                    tab.append([sec, '0', '0', '0', '0', '0', '0', '0'])
            else:
                sec = tab[-1][0]      #latest second in table
                for sec in range(tab[-1][0]+1, len(large_table)):
                    tab.append([sec, '0', '0', '0', '0', '0', '0', '0'])

    return create_table(tab1,tab2,tab3)

In [7]:
result_table_BORIS = csv_files_reader('C:/Users/oskik/InżynierkaSandbox/S01_boris/C01/Analysis/BORIS/')

In [8]:
result_table_BORIS

[[0, 0.0, 0.0, 100.0, 0.0, 0.0, 0.0, 0.0],
 [1, 0.0, 0.0, 0.0, 100.0, 0.0, 0.0, 0.0],
 [2, 0, 0, 0, 0, 0, 0, 0],
 [3, 0, 0, 0, 0, 0, 0, 0],
 [4, 0, 0, 0, 0, 0, 0, 0],
 [5, 0, 0, 0, 0, 0, 0, 0],
 [6, 0, 0, 0, 0, 0, 0, 0],
 [7, 0, 0, 0, 0, 0, 0, 0],
 [8, 0, 0, 0, 0, 0, 0, 0],
 [9, 0, 0, 0, 0, 0, 0, 0],
 [10, 0, 0, 0, 0, 0, 0, 0],
 [11, 0, 0, 0, 0, 0, 0, 0],
 [12, 0, 0, 0, 0, 0, 0, 0],
 [13, 0, 0, 0, 0, 0, 0, 0],
 [14, 0, 0, 0, 0, 0, 0, 0],
 [15, 0, 0, 0, 0, 0, 0, 0],
 [16, 0, 0, 0, 0, 0, 0, 0],
 [17, 0, 0, 0, 0, 0, 0, 0],
 [18, 0, 0, 0, 0, 0, 0, 0],
 [19, 0, 0, 0, 0, 0, 0, 0],
 [20, 0, 0, 0, 0, 0, 0, 0],
 [21, 0, 0, 0, 0, 0, 0, 0],
 [22, 0, 0, 0, 0, 0, 0, 0],
 [23, 0, 0, 0, 0, 0, 0, 0],
 [24, 0, 0, 0, 0, 0, 0, 0],
 [25, 0, 0, 0, 0, 0, 0, 0],
 [26, 0, 0, 0, 0, 0, 0, 0],
 [27, 0, 0, 0, 0, 0, 0, 0],
 [28, 0, 0, 0, 0, 0, 0, 0],
 [29, 0, 0, 0, 0, 0, 0, 0],
 [30, 0, 0, 0, 0, 0, 0, 0],
 [31, 0, 0, 0, 0, 0, 0, 0],
 [32, 0, 0, 0, 0, 0, 0, 0],
 [33, 0, 0, 0, 0, 0, 0, 0],
 [34, 0, 0, 0, 0, 0, 0, 0]

In [11]:
df = pd.DataFrame(result_table_BORIS)

In [12]:
df.to_csv('C:/Users/oskik/InżynierkaSandbox/S01_boris/C01/Analysis/BORIS/results1.csv', index=False)

In [31]:
main_directory = 'C:/Users/oskik/InżynierkaSandbox/'
target_directory_BORIS = 'Analysis/BORIS/'

s_values = ["S01", "S02", "S03", "S04", "S05", "S06", "S07", "S08", "S09", "S10", "S11" ]
c_values = ["C01/", "C02/", "C03/"]
#c_values = ["C01", "C02", "C03", "C04", "C05", "C06", "C07", "C08", "C09", "C10", "C11", "C12", "C13" ]
columns = ['Time', 'Contempt', 'Happy', 'Sad', 'Scared', 'Disgusted', 'Surprised', 'Angry']
one_zero_values = []
sum_values = 0

for c_value in c_values:
    base_path_BORIS = os.path.join(main_directory, "S01_boris/", c_value, target_directory_BORIS)
    result_table_BORIS = csv_files_reader(base_path_BORIS)
    df = pd.DataFrame(result_table_BORIS, columns=columns)
    save_path_BORIS = os.path.join(base_path_BORIS, 'results')
    df.to_csv(save_path_BORIS, index=False)

C:/Users/oskik/InżynierkaSandbox/S01_boris/C01/Analysis/BORIS/results
C:/Users/oskik/InżynierkaSandbox/S01_boris/C02/Analysis/BORIS/results
C:/Users/oskik/InżynierkaSandbox/S01_boris/C03/Analysis/BORIS/results
